# Mixed models for repeated records

This is a frequent case of longitudinal data: repeated observations
taken on individuals. The objective is to study the change of the target
variable over time (multiple measurements) and the factors (explanatory
variables) that influence this change. Observations that belong to the
same individual (patient, animal, plant, group) tend to be more similar
than observations from different individuals, and this covariance need
to be accounted for in the model of analysis.

Repeated-record data are a special case of hierarchical data, where
observations are nested within levels (e.g. milk yield measurements
within cow, or cows within herd, fish catches within region etc.).

-   $y_{ij}$: target variable measured on individual *i* at time *j*
-   $\Sigma$: symmetric covariance matrix between individuals

## Cow data

Dataset on dairy cows:

-   NID: cow ID
-   dtn: birthdate
-   dtp: calving date
-   dtcf: milk testing day
-   aua: herd
-   nl: parity
-   milk: kg/day
-   fat %
-   protein %
-   SCC: somatic cells count
-   fat kg
-   protein kg

In [ ]:
import numpy as np ## arrays
import pandas as pd ## dataframes
import seaborn as sns ## plots
import statsmodels.api as sm ## statistical models
import matplotlib.pyplot as plt ## plots

In [ ]:
url="https://raw.githubusercontent.com/filippob/longitudinal_data_analysis/refs/heads/main/data/cows/esempio.csv"
cows = pd.read_csv(url)

cows

Repeated records per individual (cow):

In [ ]:
cows['NID'].value_counts().head()

In [ ]:
cows['NID'].value_counts().value_counts().sort_index(ascending=False)

### Preprocessing

We encode dates as date data (not strings); in *Python* we can use the `Pandas` function `to_datetime` to convert strings to dates (specifying the input date format):

In [ ]:
cows.dtypes

In [ ]:
cows['date'] = pd.to_datetime(cows['dtcf'], format='%d/%m/%Y')
cows['date']

Few cows with late parities, hence we group them:

In [ ]:
cows['nl'].value_counts()

In [ ]:
# Define the breaks and labels
bins = [0, 1, 2, 3, 4, 5, np.inf]
labels = ["1", "2", "3", "4", "5", "6+"]

# Apply cut to create 'parity' column
cows['parity'] = pd.cut(cows['nl'], bins=bins, labels=labels, right=True)

In [ ]:
cows['parity'].value_counts()

Then we select the variables of interest.

-   target is milk kg / day
-   time is the test-day date
-   systematic effects are herd and parity (regrouped as above)

In [ ]:
cows_reduced = cows[['NID','date','aua','parity','latte']]
cows_reduced.rename(columns={'aua': 'herd', 'latte': 'milk'}, inplace=True)
cows_reduced

## EDA

In [ ]:
plt.figure(figsize=(8, 6))

# Boxplot with fill by parity
sns.boxplot(data=cows_reduced, x='parity', y='milk', hue='parity', dodge=False, palette='pastel')

# Jittered data points
sns.stripplot(data=cows_reduced, x='parity', y='milk',
              color='black', alpha=0.5, jitter=0.2)

# Remove legend (similar to guides(fill = "none"))
plt.legend([],[], frameon=False)

# Labels
plt.xlabel("")
plt.ylabel("Milk yield, kg/day")
plt.title("")

plt.tight_layout()
plt.show()

##### Individual cow plots

In [ ]:
## to suppress warnings with plots

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Convert NID to string to treat it as a categorical variable (like factor in R)
cows_reduced['NID'] = cows_reduced['NID'].astype(str)

# Create the FacetGrid
g = sns.FacetGrid(cows_reduced, col="NID", col_wrap=4, height=3, sharey=False)

# Map both points and lines
g.map_dataframe(sns.lineplot, x="date", y="milk", hue="NID", legend=False)
g.map_dataframe(sns.scatterplot, x="date", y="milk", hue="NID", legend=False)

# Rotate x-axis labels
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(90)

# Set axis labels
g.set_axis_labels("Test day", "Milk kg/d")

# Tight layout
plt.tight_layout()
plt.show()

By herd:

In [ ]:
cows_reduced['NID'] = cows_reduced['NID'].astype(str)
cows_reduced['herd'] = cows_reduced['herd'].astype(str)

# Set up FacetGrid by herd
g = sns.FacetGrid(cows_reduced, col="herd", col_wrap=6, height=3, sharey=False)

# Map cow-level lines
g.map_dataframe(
    sns.lineplot,
    x="date",
    y="milk",
    hue="NID",
    units="NID",
    estimator=None,
    lw=0.7,
    legend=False
)

# Rotate x-axis labels and adjust font size
for ax in g.axes.flat:
    ax.tick_params(axis='x', labelrotation=90, labelsize=6)

# Set axis labels
g.set_axis_labels("Test day", "Milk, kg/d")

plt.tight_layout()
plt.show()


### Repeatability model

$$
\mathbf{y} = \mathbf{Xb} + \mathbf{Wpe} + \mathbf{e}
$$

We have here a **permanent environment effect** (**pe**) that is linked to the fact that we have repeated measurements on the same subjects. These repeated measurements share a covariance linked to the subject they belong to.

This covariance translates to a component of the total variance of the target variable:

$$
Var(y) = Var(pe) + Var(e)
$$

Compared to simpler linear models, here we have one additional variance component besides the residual variance (i.e. variance of the target variable adjusted for the systematic effects).

1. estimate variance components and repeatability
2. get predictions of individual records (fitted values)

In [ ]:
# Ensure correct data types
cows_reduced['parity'] = cows_reduced['parity'].astype('category')
cows_reduced['herd'] = cows_reduced['herd'].astype('category')
cows_reduced['NID'] = cows_reduced['NID'].astype('category')

# Mixed Effects Model
md = sm.formula.mixedlm("milk ~ parity + herd", cows_reduced,
                        groups=cows_reduced["NID"]
                        #re_formula="~parity"  # Random intercepts for NID and random slope for parity
)
# Fit the model
mdf = md.fit()

# Print the summary
#print(mdf.summary())

In [ ]:
# Covariance matrix of random effects
print("\nCovariance of Random Effects:")
print(mdf.cov_re)

# Residual variance (residual error term)
residual_variance = mdf.scale
print("\nResidual Variance (Residual Error):")
print(residual_variance)

In [ ]:
# Extract variance components for random effects (V1)
random_effect_variance = mdf.cov_re.iloc[0, 0]  # Variance of random intercept (NID)

# Extract residual variance (V2)
residual_variance = mdf.scale  # Residual variance

# Calculate repeatability: Repeatability = V1 / (V1 + V2)
repeatability = random_effect_variance / (random_effect_variance + residual_variance)

print(f"Repeatability: {round(repeatability,3)}")

#### Model coefficients

Systematic part of the mixed model:

In [ ]:
mdf.params

#### Random effects

As many random effects as there are individuals (cows):

In [ ]:
len(mdf.random_effects)

Random effects are centered and approximately normally distributed:

In [ ]:
x = np.array([v for k,v in mdf.random_effects.items()])
x.mean()

In [ ]:
x.std()

In [ ]:
plt.hist(x)

##### Fitted values

From the model, we can obtain fitted values as:

$$
\hat{y} = \mu + \text{parity} + \text{herd} + \text{u} = \mathbf{Xb} + \mathbf{Zu}
$$

In [ ]:
y_hat = mdf.fittedvalues

There are as many fitted values as records (repeated) in the dataset:

In [ ]:
len(y_hat)

In [ ]:
plt.hist(y_hat)

Correlation between fitted and observed values.

In [ ]:
plt.scatter(cows_reduced['milk'], y_hat, alpha=0.5)
plt.show()

In [ ]:
np.corrcoef(cows_reduced['milk'], y_hat)

In [ ]:
##sqrt(sum((cows_reduced$milk-y_hat)^2)/nrow(cows_reduced)) ## RMSE
rmse = np.sqrt(((cows_reduced['milk'] - y_hat)**2).sum()/cows_reduced.shape[0])

print(round(rmse, 4))

In [ ]:
print("RMSE is", round(rmse/cows_reduced['milk'].mean()*100,3), "% of the average milk production")

The correlation between observed and predicted (fitted) values of the
target variable is one way to measure the **predictive ability** of the
model (and so is the RMSE)

**Q: have we measured correctly the predictive ability of the model?**

### (Cross) Validation

In [ ]:
# Ensure default integer index if needed
y_trn = cows_reduced.reset_index(drop=True).copy()

# Total number of rows
n = len(y_trn)

# Sample 10% of row **positions**
sample_indices = np.random.choice(n, size=int(n / 10), replace=False)

# Set milk to NaN in those rows using iloc
y_trn.iloc[sample_indices, y_trn.columns.get_loc('milk')] = np.nan

In [ ]:
y_trn

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cows_reduced, test_size=0.1, stratify = cows_reduced['NID'])

In [ ]:
train.shape

In [ ]:
# Mixed Effects Model
md = sm.formula.mixedlm("milk ~ parity + herd", train,
                        groups=train["NID"]
                        #re_formula="~parity"  # Random intercepts for NID and random slope for parity
)
# Fit the model
mdf = md.fit()

$$
\hat{y} = \mu + \text{parity} + \text{herd} + NID
$$

First, manually for the second test record:

In [ ]:
test.iloc[1,:]

<u>Systematic effects</u>:

In [ ]:
mdf.params.head()

In [ ]:
mu = np.array(mdf.params.filter(like='Intercept', axis=0)).item()
parity = np.array(mdf.params.filter(like='parity[T.2]', axis=0)).item()
herd = np.array(mdf.params.filter(like='2774304', axis=0)).item()

print("mu:", mu, "parity:", parity, "herd:", herd)

<u>Random effects</u>:

In [ ]:
rand_eff = mdf.random_effects
len(rand_eff)

In [ ]:
nid = np.array([v for k,v in rand_eff.items() if k == "V27"]).item()
print('NID effect is:', nid)

We now have all the elements to make our prediction:

In [ ]:
pred = mu + parity + herd + nid
print("The prediction for the second test record is:", round(pred, 3))

---

Now we use `predict()` to make predictions automatically on the entire test set:

In [ ]:
test_d = test.drop('milk', axis=1)

In [ ]:
test_d.head()

In [ ]:
predictions = mdf.predict(test_d)
predictions.rename("y_hat", inplace=True)
predictions.head()

In [ ]:
test = pd.merge(test, predictions, left_index=True, right_index=True)

In [ ]:
test.head()

#### Model evaluation

In [ ]:
sns.scatterplot(x="milk", y="y_hat", data=test)

In [ ]:
np.corrcoef(test['milk'], test['y_hat'])

-----------------------------------------------------------------------

**Q: The correlation between observed and predicted milk production is lower compared to the one that we measured before: why do you think it is so?**

-----------------------------------------------------------------------

Let's calculate other metrics of model performance:

In [ ]:
## function to claculate RMSE

def rmse(y, y_hat):
  n = len(y)
  squared_diff = (y-y_hat)**2
  sumsq = np.sum(squared_diff)
  mse = sumsq/n
  rmse = np.sqrt(mse)

  return rmse

In [ ]:
y = np.array(test['milk'])
y_hat = np.array(test['y_hat'])

rmseval = rmse(y,y_hat)
print(round(rmseval, 3))

And in the training set?

In [ ]:
train_d = train.drop('milk', axis=1)

In [ ]:
predictions = mdf.predict(train_d)
predictions.rename("y_hat", inplace=True)

In [ ]:
train = pd.merge(train, predictions, left_index=True, right_index=True)

In [ ]:
sns.scatterplot(x="milk", y="y_hat", data=train)

In [ ]:
np.corrcoef(train['milk'], train['y_hat'])

In [ ]:
y = np.array(train['milk'])
y_hat = np.array(train['y_hat'])

rmseval = rmse(y,y_hat)
print(round(rmseval, 3))

---

## Exercise: can you improve the model?


In [ ]:
## your code here